In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# source : https://catboost.ai/en/docs/


# # some of the columns have binary data and some have continuous (float values ) in it , we are going to categorise them in binary and continuous

In [ ]:
# binary_feature = []
# continuous_feature = [] 
# for feature in features:
#     if "int" in str(train_df[feature].dtype):   
#         binary_feature.append(feature)
#     else:
#         continuous_feature.append(feature)
# print('binary features' , binary_feature , '\n')
# print('continuous_feature' , continuous_feature) 

In [ ]:
# check count of conti feature col and binary features col
#len(binary_feature) , len(continuous_feature )

# #new code

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
dataset = pd.read_csv('/kaggle/input/tabular-playground-series-oct-2021/train.csv' )
testset = pd.read_csv('/kaggle/input/tabular-playground-series-oct-2021/test.csv' )  # we can reduce the size of 
                                                                                     #data by selecting 1000- rows only by adding parameter as
                                                                                    # nrows=10000

In [ ]:
dataset.head()

In [ ]:
testset.head()

# kde plot
# displot

In [ ]:
X1 = dataset.drop('target', axis=1)
cols = ['f10', 'f30', 'f50','f70' , 'f90', 'f110', 'f130', 'f150', 'f170' , 'f190','f210' ,'f230' ,'f250','f270','f280']
X1 = X1[cols]


lets check how the feature value is deviating 

In [ ]:
import seaborn as sns
import random
import os
import plotly.figure_factory as ff

In [ ]:
X_data = [X1.f10,  X1.f30,  X1.f50,  X1.f70,  X1.f90,  X1.f110,  X1.f130,  X1.f150, X1.f170 ,  X1.f190 , X1.f210,  X1.f230,  X1.f250,  X1.f270, X1.f280 ]
label = X1.columns.to_list()
fig = ff.create_distplot(X_data, label, bin_size=0.3, show_hist=False, show_rug=False)
fig.show()

# kde conclusion : features are not in a same proportion there is so much variation in it, so we have to select all the features in terms of geting high accuracy

In [ ]:
pip install catboost


In [ ]:
import statsmodels.api as sm
import catboost as cb
from catboost import CatBoostClassifier
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
dataset.shape #, target.shape

In [ ]:
target = dataset['target']
feature = dataset.drop(columns=['id','target'])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(feature,target, test_size = 0.3, random_state = 60,shuffle=True)
print(len(X_train))
print(len(X_test))

In [ ]:
cat_params = {
    'iterations': 15585, 
    'objective': 'CrossEntropy', 
    'bootstrap_type': 'Bernoulli',
    'learning_rate': 0.023575206684596582, 
    'reg_lambda': 36.30433203563295, 
    'random_strength': 43.75597655616195, 
    'depth': 8, 
    'min_data_in_leaf': 11, 
    'leaf_estimation_iterations': 1, 
    'subsample': 0.8227911142845009,
    #'task_type' : 'GPU',
    'eval_metric' : 'AUC',
    'verbose' : 1000,
    'early_stopping_rounds' : 500,
}

In [ ]:
cat = CatBoostClassifier(**cat_params )

cat.fit(X_train, Y_train)

test_predict = cat.predict(X_test)

In [ ]:
test_predict

In [ ]:
print('hello')

In [ ]:
# graph
res = cat.calc_feature_statistics(feature,
                                    target,
                                    feature=2,
                                    plot=True)

In [ ]:
import sklearn.metrics as sm
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

In [ ]:
y_pred_proba = cat.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(Y_test,  y_pred_proba)
auc = metrics.roc_auc_score(Y_test, y_pred_proba)
print(auc)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
testid = testset['id']
testset= testset.drop(columns=['id']) 

In [ ]:
y_prdict =cat.predict_proba(testset)[:, 1]

In [ ]:
submission = pd.DataFrame(list(zip(testid, y_prdict)),
               columns =['id', 'target'])
submission.head(20)

In [ ]:
submission.to_csv("submission.csv" , index=False )